In [3]:
import findspark
findspark.init()

In [4]:

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import DateType, TimestampType, IntegerType, FloatType, LongType, DoubleType,StringType
from pyspark.sql.types import StructType, StructField
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [5]:
spark = SparkSession.builder \
       .master("local") \
       .enableHiveSupport() \
       .appName("Spark ML") \
       .getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)


In [6]:
Custom_schema = StructType([StructField('_c0',StringType()),
                           StructField('_c1', StringType(), True),
                           StructField('_c2', StringType(), True),
                           StructField('_c3', StringType(), True),
                           StructField('_c4', StringType(), True),
                           StructField('_c5', IntegerType(), True),
                            StructField('_c6', StringType(), True),
                           StructField('_c7', IntegerType(), True),
                           StructField('_c8', IntegerType(), True),
                           StructField('_c9', IntegerType(), True),
                           StructField('_c10', IntegerType(), True),
                            StructField('_c11', IntegerType(), True)]
                            )




In [7]:

df = spark.read.csv('abc.csv', header=True, schema=Custom_schema,  sep=',')
df.head()
df = (df
   .withColumnRenamed('_c0','Registrar')
   .withColumnRenamed('_c1', 'agency')
   .withColumnRenamed('_c2', 'State')
   .withColumnRenamed('_c3', 'District')
    .withColumnRenamed('_c4', 'SubDistrict')
    .withColumnRenamed('_c5','PinCode')
   .withColumnRenamed('_c6', 'Gender')
   .withColumnRenamed('_c7', 'Age')
   .withColumnRenamed('_c8', 'generated')
    .withColumnRenamed('_c9', 'rejected')
    .withColumnRenamed('_c10', 'email')
    .withColumnRenamed('_c11', 'mobile')
   
      
    )


In [8]:
type(df)

pyspark.sql.dataframe.DataFrame

In [9]:
df.printSchema()


root
 |-- Registrar: string (nullable = true)
 |-- agency: string (nullable = true)
 |-- State: string (nullable = true)
 |-- District: string (nullable = true)
 |-- SubDistrict: string (nullable = true)
 |-- PinCode: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- generated: integer (nullable = true)
 |-- rejected: integer (nullable = true)
 |-- email: integer (nullable = true)
 |-- mobile: integer (nullable = true)



In [10]:
df.registerTempTable("data")
total_no_states = sqlContext.sql("SELECT COUNT (DISTINCT State) FROM data").collect()
total_no_males = sqlContext.sql("SELECT COUNT(Gender) FROM data WHERE Gender!='F' AND Gender!='T'").collect()
total_no_females =sqlContext.sql("SELECT COUNT(Gender) FROM data WHERE Gender!='M' and Gender!='T'").collect()
total_no_other= sqlContext.sql("SELECT COUNT(Gender) FROM data WHERE Gender!='F' AND Gender!='M'").collect()


each_state_male = sqlContext.sql("SELECT COUNT(Age),state FROM data WHERE GENDER =='M' GROUP BY state").collect()
each_state_female = sqlContext.sql("SELECT COUNT(Age),state FROM data WHERE GENDER =='F' GROUP BY state").collect()


In [11]:
state_name = []
state_name1=[]
total_count_male = []
total_count_female = []
for i in range(len(each_state_male)):
    total_count_male.append(each_state_male[i][0])
for j in range(len(each_state_male)):
    state_name.append(each_state_male[j][1])
    
    


for i in range(len(each_state_female)):
    total_count_female.append(each_state_female[i][0])
for j in range(len(each_state_female)):
    state_name1.append(each_state_female[j][1])

In [12]:
rejects = sqlContext.sql("SELECT State,count(rejected) as count FROM data WHERE rejected==1 group By State").collect()

count_of_rejects = sqlContext.sql("SELECT State,count(rejected) FROM data group by State").collect()

In [13]:
accepted = sqlContext.sql("SELECT State,count(generated) as count FROM data WHERE generated== 1 or generated==2 group By State").collect()

count_of_accepted = sqlContext.sql("SELECT State,sum(generated) FROM data group by state").collect()

In [14]:
count_of_max_rejection = sqlContext.sql("SELECT State,count(rejected) as c1 FROM data WHERE rejected==1 group by state order by c1 DESC LIMIT 10").collect()
dict(count_of_max_rejection)

{'Bihar': 4602,
 'West Bengal': 3900,
 'Uttar Pradesh': 3892,
 'Madhya Pradesh': 2785,
 'Gujarat': 1732,
 'Rajasthan': 1613,
 'Maharashtra': 1581,
 'Karnataka': 1286,
 'Odisha': 1035,
 'Tamil Nadu': 987}

In [15]:
sqlContext.sql("SELECT State, COUNT(*) AS num" +
              " FROM (SELECT * FROM data WHERE Gender NOT IN ('T')) AS Temp " +
            "GROUP BY State " +
            "Order By num DESC " +
            "LIMIT 3").collect()

[Row(State='Bihar', num=81776),
 Row(State='Uttar Pradesh', num=69476),
 Row(State='West Bengal', num=60483)]

In [16]:
sqlContext.sql("SELECT State, COUNT(*) AS num" +
        " FROM (SELECT * FROM data WHERE Gender =='F') AS Temp " +
        "GROUP BY State " +
        "Order By num DESC " +
        "LIMIT 3").collect()

[Row(State='Bihar', num=26957),
 Row(State='West Bengal', num=24823),
 Row(State='Uttar Pradesh', num=21623)]

In [17]:
sqlContext.sql("SELECT State, COUNT(*) AS num" +
        " FROM (SELECT * FROM data WHERE Gender =='M') AS Temp " +
        "GROUP BY State " +
        "Order By num ASC " +
        "LIMIT 3").collect()

[Row(State='Andaman and Nicobar Islands', num=2),
 Row(State='Lakshadweep', num=4),
 Row(State='Others', num=5)]

In [18]:
total_rejects=[]
total_reject_states=[]
total_count_rejects = []
percentage_of_rejects = []



for j in range(len(rejects)):
    total_rejects.append(rejects[j][1])
    total_count_rejects.append(count_of_rejects[j][1])

    
    
    
    


In [19]:
for n in range(len(rejects)):
    total_reject_states.append(rejects[n][0])
    
for i in range(len(rejects)):
    percentage_of_rejects.append((total_rejects[i]/total_count_rejects[i])*100)

In [20]:
total_accepted=[]
total_accepted_states=[]
total_count_accepted = []
percentage_of_accepted = []
for j in range(len(accepted)):
    total_accepted.append(accepted[j][1])
    total_count_accepted.append(count_of_accepted[j][1])
    
for n in range(len(accepted)):
    total_accepted_states.append(accepted[n][0])
    
for i in range(len(total_accepted)):
    percentage_of_accepted.append((total_accepted[i]/total_count_accepted[i])*100)
    

In [21]:
len(total_reject_states)

36

In [22]:
#agencies with max no of registrations
md =sqlContext.sql("SELECT Registrar, SUM(generated) AS TotalAadharCount FROM data GROUP BY Registrar ORDER BY TotalAadharCount DESC LIMIT 3").collect()


In [23]:
#no of identities generated in each state for male and females
md1 = sqlContext.sql("SELECT state, sum(generated) as TotalAadharCount FROM data WHERE Gender!='M' GROUP BY State ORDER BY TotalAadharCount DESC LIMIT 10").collect()



In [24]:
#state with maximum aadhar applicants
highest_state=sqlContext.sql("SELECT COUNT(*),State FROM data GROUP BY State ORDER BY COUNT(*) DESC LIMIT 3").collect()



In [25]:
h_states = []
total_count_states = []
for i in range(len(highest_state)):
    h_states.append(highest_state[i][0])
for j in range(len(highest_state)):
    total_count_states.append(highest_state[j][1])

In [26]:
#district with maximum aadhar applicants

highest_district=sqlContext.sql("SELECT COUNT(*), District FROM data GROUP BY District ORDER BY COUNT(*) DESC LIMIT 3").collect()



In [27]:
h_district = []
total_count_district = []
for i in range(len(highest_state)):
    h_district.append(highest_district[i][0])
for j in range(len(highest_state)):
    total_count_district.append(highest_district[j][1])

In [28]:
total_mobile_registerd = sqlContext.sql("SELECT count(mobile) as count FROM data WHERE mobile==1 or mobile==2").collect()
total_mobile_not_registerd= sqlContext.sql("SELECT count(mobile) as count FROM data WHERE mobile==0").collect()


In [29]:
total_email_registerd = sqlContext.sql("SELECT count(email) as count FROM data WHERE email==1 or email==2").collect()
total_email_not_registerd= sqlContext.sql("SELECT count(email) as count FROM data WHERE email==0").collect()


In [30]:
states = []*len(md1)
total_count1 = []*len(md1)
for i in range(len(md1)):
    states.append(md1[i][0])
for j in range(len(md1)):
    total_count1.append(md1[j][1])
    
    


In [31]:
aadhar_company = []*len(md)
total_count = []*len(md)
for i in range(len(md)):
    aadhar_company.append(md[i][0])
for j in range(len(md)):
    total_count.append(md[j][1])
    
    

In [32]:
from bokeh.io import curdoc,export_png, export_svgs
from bokeh.io import output_file, show
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.layouts import row
from bokeh.layouts import column
from math import pi
from bokeh.models.widgets import Tabs, Panel 
from bokeh.models import Button
from bokeh.models import CheckboxGroup, RadioGroup, Toggle
from bokeh.models import NumeralTickFormatter
from bokeh.models import ColumnDataSource, CDSView, GroupFilter
from bokeh.models import Slider
from bokeh.events import ButtonClick
from bokeh.models import Button,CustomJS
import math
from bokeh.io import save
from bokeh.palettes import Spectral6
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Div
from bokeh.models.widgets import Paragraph
from bokeh.models.widgets import PreText
from collections import Counter 
from bokeh.models import LayoutDOM

In [33]:
x = dict(md1)
data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
data['angle'] = data['value']/data['value'].sum() * 2*pi
data['percent'] = data['value'] / sum(x.values()) * 100
data['color'] = Category20c[len(x)]


#pie chart Distribution of Population in each state
p = figure(plot_height=400, title="Distribution of Population in Top 10 states", toolbar_location=None,
           tools="hover", tooltips=[("country", "@country"),("Percentage", "@percent{0.2f}%")], x_range=(-0.5, 1.0),sizing_mode='scale_width')

p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend='country', source=data)
p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None





#Top 3 Agencies with maximum number of Registrations
p1 = figure(x_range=aadhar_company, y_range=[0,1000000], plot_height=350, title="Top 3 Agencies with Maximum no of registrations",
           toolbar_location=None, tools="",sizing_mode='scale_width')
p1.vbar(x=aadhar_company, top= total_count, width=0.8)
p1.yaxis.formatter=NumeralTickFormatter(format="00")
p1.xgrid.grid_line_color = None
p1.y_range.start = 0


#Total Number of Males and Females in each State
p2 = figure(x_range= state_name, y_range=[0,55000], plot_height=550, title="Total Number of Males and Females in Each State",
           toolbar_location=None, tools="",sizing_mode='scale_width')
p2.xaxis.axis_label = 'States'
p2.yaxis.axis_label = 'Number of Population'

p2.vbar(x= state_name, top= total_count_male, width=0.8,fill_color = "green",legend = 'male')
p2.yaxis.formatter=NumeralTickFormatter(format="00")
p2.xgrid.grid_line_color = None
p2.y_range.start = 0
p2.xaxis.major_label_orientation = -45
p2.vbar(x= state_name1, top= total_count_female, width=0.8,fill_color ="blue",legend= 'female')
p2.yaxis.formatter=NumeralTickFormatter(format="00")
p2.xgrid.grid_line_color = None
p2.y_range.start = 0
p2.xaxis.major_label_orientation = -45



#Total Number of Males, Females and others
category = ['Male','Female','other']
val = [292798, 148013, 7]
p3 = figure(
    x_range= category, 
    y_range=[0,500000], 
    plot_height=250, 
    title="Gender Distribution",
    toolbar_location=None,
    tools="",
    sizing_mode='scale_width'
  
   
)
p3.xaxis.axis_label = 'Gender'
p3.yaxis.axis_label = 'Population Value'
p3.vbar(x= category, top = val, width=0.8,fill_color = ['blue','orange','black'])
p3.yaxis.formatter=NumeralTickFormatter(format="00")

p3.xgrid.grid_line_color = None
p3.y_range.start = 0

#Ov
p4 = figure(
  y_range= total_reject_states,
  title = '% of Rejection of Aadhar in each State',
  x_axis_label ='',
  plot_width=600,
  plot_height=500,
  tools="",
  sizing_mode='scale_width'  
 
)
p4.xaxis.axis_label = "Percentage value of Rejection of Aadhar"
p4.yaxis.axis_label = "States"
p4.hbar(
    y=total_reject_states,
    right= percentage_of_rejects,
    left=0,
    height=0.4,
    color='red',
    fill_alpha=0.5
)


p5 = figure(
  y_range= total_accepted_states,
  title ='% of Accepted of Aadhar in each State',
  x_axis_label ="",
  plot_width=600,
  plot_height=500,
  tools="",
  sizing_mode='scale_width'  
 
)
p5.xaxis.axis_label = "Percentage value of Accepted Aadhar"
p5.yaxis.axis_label = "States"
p5.hbar(
    y=total_accepted_states,
    right= percentage_of_accepted,
    left=0,
    height=0.4,
    color='green',
    fill_alpha=0.75
)

#percent of mobile registered 

val1 = Counter({
    'Yes': total_mobile_registerd[0][0], 'No': total_mobile_not_registerd[0][0]
})

data = pd.DataFrame.from_dict(dict(val1), orient='index').reset_index().rename(index=str, columns={0:'value', 'index':'country'})
data['angle'] = data['value']/sum(val1.values()) * 2*pi
data['color'] = ['#6baed6', '#e6f598',]
data['percent'] = data['value'] / sum(val1.values()) * 100
# Plotting code

p6 = figure(plot_height=350, toolbar_location=None, title ='% of total Mobile Registered',
           tools="hover",tooltips = [("Value", "@value"),("Percentage", "@percent{0.2f}%")],sizing_mode='scale_width')

p6.annular_wedge(x=0, y=1, inner_radius=0.2, outer_radius=0.4,
                start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                line_color="white", fill_color='color', legend='country', source=data)


p6.axis.axis_label=None
p6.axis.visible=False
p6.grid.grid_line_color = None


#percent of email registered
val2 = Counter({
    'Registered': total_email_registerd[0][0], 'Not Registered': total_email_not_registerd[0][0]
})

data = pd.DataFrame.from_dict(dict(val2), orient='index').reset_index().rename(index=str, columns={0:'value', 'index':'country'})
data['angle'] = data['value']/sum(val2.values()) * 2*pi
data['color'] = [ '#fc8d59', '#d53e4f']
data['percent'] = data['value'] / sum(val2.values()) * 100
# Plotting code

p7 = figure(plot_height=350, toolbar_location=None, title ='% of Accepted of Aadhar in each State',
           tools="hover",tooltips = [("Value", "@value"),("Percentage", "@percent{0.2f}%")],sizing_mode='scale_width')

p7.annular_wedge(x=0, y=1, inner_radius=0.2, outer_radius=0.4,
                start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
                line_color="white", fill_color='color', legend='country', source=data)


p7.axis.axis_label=None
p7.axis.visible= False
p7.grid.grid_line_color = None



In [37]:
div = Div(text= """<b>Analysis of a dataset from Aadhaar- a unique identity
issued to the residents of India by UIDAI</b>""", width=500, height=100,style={'font-size': '120%', 'color': 'black','text-align':'center','background-color':'#FFFF99'})

div1 = Div(text="""<i>Aadhaar is a unique identity issued to all the
residents of India by UIDAI. The dataset provided by
the authority is “Enrollments processed in detail”,
which contains following columns: Registrar,
Enrollment Agency, State, District, Sub District, Pin
Code, Age, Gender, Aadhaar Generated, Enrollment
Rejected, Email And Mobile Number.</i><br></br>""", width=500, height=200,style={'font-size': '120%', 'color': 'black','text-align':'center'})

div2 = Div(text=""" <ul>
<li>Distribution of Population in each state</li>
<li>Top 3 Agencies with maximum number of Registrations</li>
<li>Total Number of Males, Females and others</li>
<li>Percentage of Rejection of Aadhar in each State</li>
<li>Percentage of Acception of Aadhar in each State</li>
<li>Percent of Mobile Number registered</li>
<li>Percent of Email-id registered</li>


</ul>""",width=500, height=100,style={'font-size': '100%', 'color': 'black'})

div3 = Div(text=""" <ul>
<li>Distribution of Population in each state</li>
<li>Top 3 Agencies with maximum number of Registrations</li>
<li>Total Number of Males, Females and others</li>
<li>Percentage of Rejection of Aadhar in each State</li>
<li>Percentage of Acception of Aadhar in each State</li>
<li>Percent of Mobile Number registered</li>
<li>Percent of Email-id registered</li>


</ul>""",width=500, height=100,style={'font-size': '100%', 'color': 'black','margin': '-100px'})

x_range = (-10,-5) # could be anything - e.g.(0,1)
y_range = (10,20)
p9 = figure(x_range=x_range, y_range=y_range)
#img_path = 'https://bokeh.pydata.org/en/latest/_static/images/logo.png'
img_path = 'aadharlogo.png'
p9.image_url(url=[img_path],x=x_range[0],y=y_range[1],w=x_range[1]-x_range[0],h=y_range[1]-y_range[0])
p9.axis.visible = False
p9.grid.grid_line_color = None
p9.axis.axis_label=None
p9.toolbar.autohide =True

p8 = figure(x_range=x_range, y_range=y_range)
#img_path = 'https://bokeh.pydata.org/en/latest/_static/images/logo.png'
img_path = 'info.png'
p8.image_url(url=[img_path],x=x_range[0],y=y_range[1],w=x_range[1]-x_range[0],h=y_range[1]-y_range[0])
p8.axis.visible = False
p8.grid.grid_line_color = None
p8.axis.axis_label=None
p8.toolbar.autohide =True

front = Panel(child= row(p8,p9),title='0')
first =Panel(child= row(p,p1), title='1')
second = Panel(child= row(p2,p3), title='2')
third = Panel(child= row(p4,p5), title='3')
four = Panel(child=row(p6,p7),title='4')
#tabs=Tabs(tabs=[Panel(title=str(i),child = column(1,2,3))])
tabs = Tabs(tabs=[front,first, second,third,four])


curdoc().add_root(column(tabs))
output_file('tabbed.html')
show(tabs)
